# Пример генерации моделей

В данном примере показано, как с помощью программного интерфейса (API) можно обучить генеративную модель и использовать ее для генерации модели СКФС. В качестве прикладной области используются модели предприятий, основанные на публичном наборе данных SAP-SAM.

In [1]:
import sys
# Для загрузки модулей из основного каталога OrGAN,
# который находится в родительском каталоге
sys.path.insert(0, '..')

from organ.solver import Solver
from organ.config import make_config
from organ.demo import ManagementModel

In [2]:
import os
import random
import numpy as np

## Генерация моделей СКФС

Генерация выполняется с использованием модели, обученной на 175000 итераций и размещенной в каталоге 'output/models_management'.

In [20]:
config = make_config(
    rules=ManagementModel(),
    data_dir='../demo_data/management_aug',
    model_save_dir='../output/models_management',
    log_dir='../output/logs_management',
    test_iters=175000,
)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
    
solver_g = Solver(config)
orgs = solver_g.generate_valid(10, ctx=[2000,1])

Max nodes: 9
Node types: 9 9
Edge types: 7 7
CPGenerator(
  (cond_encoder): FCBlock(
    (layers): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=12, bias=True)
      (3): Tanh()
    )
  )
  (layers): FCBlock(
    (layers): Sequential(
      (0): Linear(in_features=20, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): Tanh()
      (4): Linear(in_features=256, out_features=512, bias=True)
      (5): Tanh()
    )
  )
  (edges_spec_layer): Linear(in_features=512, out_features=32, bias=True)
  (edge_layers): FCBlock(
    (layers): Sequential(
      (0): Linear(in_features=41, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): Tanh()
      (4): Linear(in_features=64, out_features=32, bias=True)
      (5): Tanh()
    )
  )
  (nodes_layer): Linear(in_features=512,

## Отображение результата генерации

In [21]:
print('Подразделения:\n', orgs[0].nodes)
for node in orgs[0].nodes:
    if node > 0:
        print("Подразделение ", node, ": ", config.rules.node_type_dict[node]["title"])
print('\nСвязи:\n', orgs[0].edges)

i = 0
for edge_row in orgs[0].edges:
    j = 0
    for edge_val in edge_row:
        if edge_val > 0:
            print("Связь (", i, ", ", j, ", тип: ", edge_val, ") от ", 
                  config.rules.node_type_dict[i]["title"], " к ", config.rules.node_type_dict[j]["title"]
                 )
        j += 1
    i += 1
    
print("\nЗначения параметров подразделений: \n", orgs[0].node_features)

Подразделения:
 [0 0 2 0 4 0 6 7 8]
Подразделение  2 :  Headquarters
Подразделение  4 :  Marketing department
Подразделение  6 :  Logistics
Подразделение  7 :  ERP System
Подразделение  8 :  Shop

Связи:
 [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 5 3 0]]
Связь ( 2 ,  7 , тип:  2 ) от  Headquarters  к  ERP System
Связь ( 4 ,  7 , тип:  2 ) от  Marketing department  к  ERP System
Связь ( 8 ,  6 , тип:  5 ) от  Shop  к  Logistics
Связь ( 8 ,  7 , тип:  3 ) от  Shop  к  ERP System

Значения параметров подразделений: 
 [[0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00]
 [2.5862387e-01 2.1219337e+00]
 [0.0000000e+00 0.0000000e+00]
 [1.4446613e+01 1.2807310e+00]
 [0.0000000e+00 0.0000000e+00]
 [1.5984881e+00 1.7089773e+00]
 [0.0000000e+00 3.7603176e+01]
 [1.9680187e+03 5.5207210e+00]]
